# DeepLabV3 Model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights

In [ ]:

NUM_CLASSES = 19  # Cityscapes
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 1e-4
WEIGHTS = DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1

model = deeplabv3_resnet50(weights=WEIGHTS)
model.classifier[4] = nn.Conv2d(256, NUM_CLASSES, kernel_size=1)
model.to(DEVICE)

criterion = nn.CrossEntropyLoss(ignore_index=255)  # 255 = unlabeled

optimizer = optim.Adam(model.parameters(), lr=LR)

def compute_iou(preds, labels, num_classes):
    preds = torch.argmax(preds, dim=1)
    ious = []
    for cls in range(num_classes):
        pred_inds = (preds == cls)
        target_inds = (labels == cls)
        intersection = (pred_inds & target_inds).sum().item()
        union = (pred_inds | target_inds).sum().item()
        if union == 0:
            ious.append(float('nan'))  # ignore class with no pixels
        else:
            ious.append(intersection / union)
    return sum([iou for iou in ious if not torch.isnan(torch.tensor(iou))]) / len(ious)

def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_iou = 0
    for images, masks in dataloader:
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_iou += compute_iou(outputs.detach(), masks, NUM_CLASSES)

    avg_loss = total_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    print(f"Loss: {avg_loss:.4f} | IoU: {avg_iou:.4f}")
    return avg_loss, avg_iou
